#  Neuralne mreže

**"A computing system made up of a number of simple, highly interconnected processing elements, which process information by their dynamic state response to external inputs"**



# Pregled
* Neuron
* Arhitektura 
* Forward propagation i backpropagation
* Odabir aktivacione funkcije
* Klasifikacija neuralnih mreža
  * Multilayer Perceptron (MLP)

## Neuralne mreže
* Model izračunavanja koji nam omogućava da definišemo kompleksnije hipoteze nego do sad
* Jedna istrenirana neuralna mreža = jedna hipoteza
  * Istrenirana = pronađene adekvatne vrednosti parametara (kao i do sad)
  * I dalje ćemo imati ulaz u vidu feature vektora $x$ 
  * I dalje ćemo imati izaz u vidu rezultata hipoteze $h_w(x)$
* Sastoje se od jednostavnih jedinica - **neurona**

## Neuron
* Jedan neuron je zapravo trojka $(w, b, f)$ - $w$ je vektor težina, $b$ je *bias* vrednost, a $f$ je **aktivaciona funkcija**
* Svaki neuron dobija ulaz (vektor $x$) i transformiše ga po formuli $f(wx+b)$ - linearna kombinacija sa biasom nad kojom je primenjena funkcija aktivacije
* 1 neuron sa sigmoidnom aktivacijom je u potpunosti ekvivalentan logističkoj regresiji!
* Napomena: Neki autori umesto odvojene vrednosti za bias dodaju još jednu vrednost u vektor težina i na svaki feature vektor dodaju novi feature čija je vrednost uvek jedan - ovo daje iste rezultate ali menja notaciju


## Arhitektura
* Sada kada znamo kako radi jedan neuron, neuralna mreža je samo kombinacija velikog broja njih
* Neuroni se kombinuju "u dve dimenzije"
* Prvo "vertikalno", gde više neurona čini jedan **sloj** mreže (više transformacija odjednom tako da se od feature vektora $x$ dobije novi feature vektor $x'$)
  * Kombinujemo parametre neurona u parametre sloja tako da dobijamo $W$ - matricu težina, $b$ - bias vektor, $f$ - jedinstvenu funkciju aktivacije za ceo sloj
* Zatim "horizontalno", gde se više slojeva niže tako da izlaz jednog postaje ulaz narednog
* Ulazni podaci čine **ulazni sloj**, poslednji sloj mreže čini **izlazni sloj**, a svi slojevi između se zovu **skriveni slojevi** (jer njihove vrednosti ne posmatramo direktno)
* Dakle, uloga svakog sloja je da transformiše feature vektor u novi feature vektor (osim izlaznog, koji će uglavnom da vrši finalna izračunavanja - npr. softmax)


* **Primer**: mreža za binarnu klasifikaciju feature vektora dimenzije 10 sa jednim skrivenim slojem sastavljenim od 3 neurona
  * Skicirati mrežu, definisati sve njene parametre, kao i hipotezu
  * $a^{(1)} = f^{(1)}(z^{(1)}) = f^{(1)}(W^{(1)}x+b^{(1)})$
  * $h = a^{(2)} = f^{(2)}(z^{(2)}) = f^{(2)}(W^{(2)}a^{(1)}+b^{(2)})$
  
## Forward propagation i backpropagation
* Proces izračunavanja kojim mreža dobija hipotezu od ulaznih feature-a naziva se **forward propagation** (propagacija unapred)
* Do sada nismo pomenuli kako treniramo mrežu tj. kako tokom treninga ažuriramo parametre
* Upravo to radi **backpropagation** (propagacija unazad) - proces koji nakon propagacije unapred računa funkciju troška i propagira njene izvode unazad kroz mrežu, koristeći ih za ažuriranje parametara na svim slojevima
  * Detalje ćemo sada preskočiti, ali ovo je ključan mehanizam neuralnih mreža i važno je razumeti kako funkcioniše

## Odabir aktivacione funkcije
* Pojam aktivacione funkcije smo videli još u primeru logističke regresije (ali tada nismo koristili ovaj naziv)
* Svrha aktivacione funkcije je da unese nelinearnost u sistem (bez nje bi mreža sa $N$ slojeva bila ekvivalentna mreži sa jednim slojem, pa ne koristimo punu moć arhitekture)
* Kandidati:
  * identitet - $f(z) = z$
  * sigmoid - $f(z) = \frac{1}{1+e^{(-z)}}$
  * tangens hiperbolički - $f(z) = tanh(z)$
  * rectified linear unit (ReLU) - $f(z) = max(0, z)$
  * leaky ReLU - $f(z) = z$ ako je $z>=0$, a $f(z)=\alpha z$ u suprotnom, gde je $\alpha$ mala unapred određena konstanta

## Klasifikacija neuralnih mreža
- Neuralne mreže koje nemaju cikluse u grafu povezanosti neurona nazivamo **feedforward mreže** (suprotno, one koje imaju cikluse zovemo **rekurentne mreže**)
- Mreže koje smo do sada definisali su isključivo feedforward
- Najprostija feedforward mreža je **perceptron**: 0 skrivenih slojeva
- Ukoliko imamo 1 ili više skrivenih slojeva takvu mrežu zovemo **multilayer perceptron (MLP)**
  - MLP ćemo implementirati za MNIST problem na tri nivoa apstrakcije: 
    - Prvo, koristeći samo numpy, dakle bez specijalizovanih biblioteka
    - Zatim, koristeći TensorFlow
    - Zatim, koristeći Keras

## Dodatni resursi

http://cs231n.github.io/neural-networks-1/


http://neuralnetworksanddeeplearning.com/


http://karpathy.github.io/neuralnets/


http://ufldl.stanford.edu/tutorial/supervised/MultiLayerNeuralNetworks/

## Implementacija MLP uz numpy bez TensorFlow
[Link](https://github.com/rand0musername/psiml2017-workshops/blob/master/2%20-%20NN/code/nn.py)

## Implementacija MLP u TensorFlow

In [16]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline

# Ucitavamo MNIST skup podataka.
mnist = tf.keras.datasets.mnist
(train_x, train_y), (test_x, test_y) = mnist.load_data()

nb_train = len(train_y)
nb_test = len(test_y)
#print(nb_train, nb_test)
#print(train_y)

# Prikazujemo prvu sliku.
#plt.imshow(train_x[0], cmap='Greys')  

train_x = np.reshape(train_x, [nb_train, -1])
test_x = np.reshape(test_x, [nb_test, -1])

# Parametri mreze
learning_rate = 0.001
nb_epochs = 16
batch_size = 128

# Parametri arhitekture
nb_input = 28*28    # MNIST data input (img shape: 28*28)
nb_hidden1 = 256 # 1st layer number of neurons
nb_hidden2 = 256 # 2nd layer number of neurons
nb_classes = 10   # MNIST total classes (0-9 digits)

# Sama mreza
X = tf.placeholder(shape=(None, nb_input), dtype=tf.float32)
Y = tf.placeholder(shape=(None), dtype=tf.int32)
Y_onehot = tf.one_hot(Y, nb_classes)

w = {
    '1': tf.Variable(tf.random_normal([nb_input, nb_hidden1])),
    '2': tf.Variable(tf.random_normal([nb_hidden1, nb_hidden2])),
    'out': tf.Variable(tf.random_normal([nb_hidden2, nb_classes]))
}

b = {
    '1': tf.Variable(tf.random_normal([nb_hidden1])),
    '2': tf.Variable(tf.random_normal([nb_hidden2])),
    'out': tf.Variable(tf.random_normal([nb_classes]))
}

f = {
    '1': tf.nn.relu,
    '2': tf.nn.relu,
    'out': tf.nn.softmax
}

z1 = tf.add(tf.matmul(X, w['1']), b['1'])
a1 = f['1'](z1)
z2 = tf.add(tf.matmul(a1, w['2']), b['2'])
a2 = f['2'](z2)
z_out = tf.add(tf.matmul(a2, w['out']), b['out']) # a2 ovde!
pred = f['out'](z_out)

pred_correct = tf.equal(tf.argmax(pred, 1), tf.argmax(Y_onehot, 1))
accuracy = tf.reduce_mean(tf.cast(pred_correct, tf.float32))

loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=z_out, labels=Y_onehot))

# GD je djubre
opt_op = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)

#print(train_x.shape)

# Trening!
with tf.Session() as sess:
  sess.run(tf.global_variables_initializer())

  for epoch in range(nb_epochs):
    epoch_loss = 0
    nb_batches = int(nb_train / batch_size)
    for i in range(nb_batches):
      feed = {X: train_x[i*batch_size : (i+1)*batch_size, :], 
              Y: train_y[i*batch_size : (i+1)*batch_size]}
      _, curr_loss = sess.run([opt_op, loss], feed_dict=feed)

      epoch_loss += curr_loss

    # U svakoj desetoj epohi ispisujemo prosečan loss.
    epoch_loss /= nb_train
    print('Epoch: {}/{}| Avg loss: {:.5f}'.format(epoch+1, nb_epochs, epoch_loss))
    
  # Test!
  feed = {X: test_x, Y: test_y}
  test_acc = sess.run(accuracy, feed_dict = feed)
  print('Test acc: {}'.format(test_acc))

Epoch: 1/16| Avg loss: 416.39657
Epoch: 2/16| Avg loss: 97.68244
Epoch: 3/16| Avg loss: 63.09391
Epoch: 4/16| Avg loss: 45.20441
Epoch: 5/16| Avg loss: 33.77310
Epoch: 6/16| Avg loss: 25.81210
Epoch: 7/16| Avg loss: 19.98838
Epoch: 8/16| Avg loss: 15.65351
Epoch: 9/16| Avg loss: 12.04712
Epoch: 10/16| Avg loss: 9.36258
Epoch: 11/16| Avg loss: 7.29019
Epoch: 12/16| Avg loss: 5.85051
Epoch: 13/16| Avg loss: 4.40729
Epoch: 14/16| Avg loss: 3.58028
Epoch: 15/16| Avg loss: 2.85545
Epoch: 16/16| Avg loss: 2.18576
Test acc: 0.9433000087738037


## Implementacija MLP u biblioteci Keras

In [14]:
import numpy as np
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import matplotlib.pyplot as plt
%matplotlib inline

mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(256, activation=tf.nn.relu),
  #tf.keras.layers.Dense(256, activation=tf.nn.relu),
  #tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10, activation=tf.nn.softmax)
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(x_train, y_train, epochs=16)

model.evaluate(x_test, y_test)

Epoch 1/16
60000/60000 [==============================] - 8s 129us/sample - loss: 0.2253 - acc: 0.9356
Epoch 2/16
60000/60000 [==============================] - 8s 127us/sample - loss: 0.0921 - acc: 0.9718
Epoch 3/16
60000/60000 [==============================] - 8s 128us/sample - loss: 0.0609 - acc: 0.9815
Epoch 4/16
 6208/60000 [==>...........................] - ETA: 6s - loss: 0.0378 - acc: 0.9889

KeyboardInterrupt: ignored